In [ ]:
!pip install vantage6-client pyvis

In [ ]:
# Setup, run once
from vantage6.client import Client
from uuid import uuid4

In [ ]:
client = Client("http://localhost", 5000, "/api")
client.authenticate("node1-admin", "password")
client.setup_encryption(None)

In [ ]:
# Find out the organizations and collaborations we can run on
import json
collaboration_list = client.collaboration.list()
collaboration_index = 0
organization_ids_ = [ ]

for organization in collaboration_list[collaboration_index]['organizations']:
    organization_ids_.append(organization['id'])
print(json.dumps(client.node.list(), indent=2))

In [ ]:
# Request the structure to be built
input_ = {
    "master": "true",
    "method":"master", 
    "args": [ ],
    "kwargs": {}
}

task = client.post_task(
    name="GetGraphStructure",
    image="jaspersnel/v6-graphstructure-rdf-py:swat4ls-0.1.0",
    collaboration_id=collaboration_list[collaboration_index]['id'],#Get the first collaboration associated with user
    input_= input_,
    organization_ids=[organization_ids_[0]]
)

print(json.dumps(task, indent=2))

In [ ]:
# Wait for results
import time
import json
resultObjRef = task.get("results")[0]
resultObj = client.result.get(resultObjRef['id'])
attempts = 1
while((resultObj["finished_at"] == None) and attempts < 10):
    print("waiting...")
    time.sleep(5)
    resultObj = client.result.get(resultObjRef['id'])
    attempts += 1
results = resultObj['result']
intersect = results['intersect']
union = results['union']
uri_data = results['uri_data']
print(intersect[:10])
print(union[:10])

In [ ]:
# List (some of) the classes we've found
classes = set()
for s, p, o in union + intersect:
    classes.add(s)
    classes.add(o)

for i, cls in enumerate(classes):
    print(cls)
    if i >= 9:
        break

In [ ]:
# Generate namespaces
import re

namespaces = set()

# Extract all the namespaces by removing everything after the last / or #
for uri in classes.union(set([trp[1] for trp in union])):
    namespace = re.sub(r'(.+[#/])[^#/]+', r'\1', uri)
    namespaces.add(namespace)

# Take the last part of the URI to be the prefix
# (probably too naive to generalize well, could build in checks)
namespaces = {ns: re.split(r'[#/]', ns)[-2] for ns in namespaces}

for ns, pref in namespaces.items():
    print(f'{ns} - {pref}')

def transform_prefix(uri: str):
    for ns, pref in namespaces.items():
        if uri.startswith(ns):
            return uri.replace(ns, pref + ':')
    return uri

def get_label(uri: str):
    return uri_data.get(uri, {}).get('label', {})

In [ ]:
# Show all the common elements
from pyvis.network import Network

intersect_graph = Network(notebook=True, directed=True)

for s, p, o in intersect:
    titles = [
        f'{get_label(el)}\n{transform_prefix(el)}' if get_label(el) else transform_prefix(el)
        for el in [s, o, p]
    ]
    to_node = o if uri_data[o]['type'] == 'class' else o + str(uuid4())
    intersect_graph.add_nodes(
        [s, to_node], 
        label=titles[:2])
    intersect_graph.add_edge(s, to_node, title=titles[2])

intersect_graph.show('output/intersect.html')

In [ ]:
# Show all elements
union_graph = Network(notebook=True, directed=True)

for s, p, o in union:
    titles = [
        f'{get_label(el)}\n{transform_prefix(el)}' if get_label(el) else transform_prefix(el)
        for el in [s, o, p]
    ]
    to_node = o if uri_data[o]['type'] == 'class' else o + str(uuid4())
    union_graph.add_nodes(
        [s, to_node], 
        label=titles[:2])
    union_graph.add_edge(s, to_node, title=titles[2])

union_graph.show('output/union.html')

In [ ]:
# Combine them!
graph = Network(notebook=True, directed=True)

intersect_classes = [s for s, _, _ in intersect] + [o for _, _, o in intersect]

for s, p, o in union:
    titles = [
        f'{get_label(el)}\n{transform_prefix(el)}' if get_label(el) else transform_prefix(el)
        for el in [s, o, p]
    ]

    color = 'pink'
    if s in intersect_classes:
        color = 'lightblue'
    graph.add_node(s, color=color, label=titles[0])

    to_node = o if uri_data[o]['type'] == 'class' else o + str(uuid4())

    color = 'pink'
    if o in intersect_classes:
        color = 'lightblue'
    if uri_data[o]['type'] == 'literal':
        color = 'grey'

    graph.add_node(to_node, color=color, label=titles[1])

    graph.add_edge(s, to_node, title=titles[2])

graph.show('output/integrated.html')